In [1]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import xarray
import pandas as pd

In [3]:
from mmctools.dataloaders import read_dir
from mmctools.wrf.utils import extract_column_from_wrfdata
from mmctools.coupling.internal import timeheight_to_sowfa, ICs_to_sowfa, BCs_to_sowfa

Define some physical constants

In [4]:
g  = 9.81            # Gravity [m s-2]
T0 = 300             # Reference temperature for perturbation temperature [K]
K  = 0.41            # von Karman constant
R_air = 287.058      # Specific gas constant for dry air [J kg-1 K-1]
Cp_air = 1005        # Specific heat of air [J kg-1 K-1]
P0 = 100000          # Reference pressure [Pa]
kappa = R_air/Cp_air # Poisson constant

In [5]:
# TODO's
# ------
# - rescaling with MU* after spatial filtrering means we are using the average MU*, is this what we want?
# - some arrays are float 32, some are float64. Decide on what precision we are going to use
# - make coupling.internal routines compatible with xarrays

# Extract WRF data at a specific site and write as internal forcing for SOWFA

Input files

In [6]:
dpath = '/projects/mmc/SWIFTRegion/8Nov2013/WRF_144hrs/WRFdata'
file_filter  = 'wrfout_d03_*'

SWIFT site coordinates

In [7]:
site_latitude  = 33.61054
site_longitude = -102.05054

Some parameters for extracting site data

In [8]:
# Define how WRF data is filtered to site specific data
# - 'interpolate': interpolate to site coordinates
# - 'nearest': use nearest WRF grid point
# - 'average': average of an area L_filter x Lfilter centred around the site
spatial_filter = 'nearest'
L_filter = 0.0

# Define microscale vertical grid
Ztop = 2000.0 # Column height [m]
Vres = 5.0    # Vertical resolution [m]

## Read WRF data

In [9]:
%%time
xa = read_dir(dpath, file_filter=file_filter,
              reader=extract_column_from_wrfdata,
              verbose=True,
              coords=(site_latitude,site_longitude),
              spatial_filter=spatial_filter,L_filter=L_filter,
              Ztop=Ztop,Vres=Vres,
             )

Reading /projects/mmc/SWIFTRegion/8Nov2013/WRF_144hrs/WRFdata/wrfout_d03_2013-11-06_12:00:00
Reading /projects/mmc/SWIFTRegion/8Nov2013/WRF_144hrs/WRFdata/wrfout_d03_2013-11-06_13:00:00
Reading /projects/mmc/SWIFTRegion/8Nov2013/WRF_144hrs/WRFdata/wrfout_d03_2013-11-06_14:00:00
Reading /projects/mmc/SWIFTRegion/8Nov2013/WRF_144hrs/WRFdata/wrfout_d03_2013-11-06_15:00:00
Reading /projects/mmc/SWIFTRegion/8Nov2013/WRF_144hrs/WRFdata/wrfout_d03_2013-11-06_16:00:00
Reading /projects/mmc/SWIFTRegion/8Nov2013/WRF_144hrs/WRFdata/wrfout_d03_2013-11-06_17:00:00
Reading /projects/mmc/SWIFTRegion/8Nov2013/WRF_144hrs/WRFdata/wrfout_d03_2013-11-06_18:00:00
Reading /projects/mmc/SWIFTRegion/8Nov2013/WRF_144hrs/WRFdata/wrfout_d03_2013-11-06_19:00:00
Reading /projects/mmc/SWIFTRegion/8Nov2013/WRF_144hrs/WRFdata/wrfout_d03_2013-11-06_20:00:00
Reading /projects/mmc/SWIFTRegion/8Nov2013/WRF_144hrs/WRFdata/wrfout_d03_2013-11-06_21:00:00
Reading /projects/mmc/SWIFTRegion/8Nov2013/WRF_144hrs/WRFdata/wrfout_d

Reading /projects/mmc/SWIFTRegion/8Nov2013/WRF_144hrs/WRFdata/wrfout_d03_2013-11-10_05:00:00
Reading /projects/mmc/SWIFTRegion/8Nov2013/WRF_144hrs/WRFdata/wrfout_d03_2013-11-10_06:00:00
Reading /projects/mmc/SWIFTRegion/8Nov2013/WRF_144hrs/WRFdata/wrfout_d03_2013-11-10_07:00:00
Reading /projects/mmc/SWIFTRegion/8Nov2013/WRF_144hrs/WRFdata/wrfout_d03_2013-11-10_08:00:00
Reading /projects/mmc/SWIFTRegion/8Nov2013/WRF_144hrs/WRFdata/wrfout_d03_2013-11-10_09:00:00
Reading /projects/mmc/SWIFTRegion/8Nov2013/WRF_144hrs/WRFdata/wrfout_d03_2013-11-10_10:00:00
Reading /projects/mmc/SWIFTRegion/8Nov2013/WRF_144hrs/WRFdata/wrfout_d03_2013-11-10_11:00:00
Reading /projects/mmc/SWIFTRegion/8Nov2013/WRF_144hrs/WRFdata/wrfout_d03_2013-11-10_12:00:00
Reading /projects/mmc/SWIFTRegion/8Nov2013/WRF_144hrs/WRFdata/wrfout_d03_2013-11-10_13:00:00
Reading /projects/mmc/SWIFTRegion/8Nov2013/WRF_144hrs/WRFdata/wrfout_d03_2013-11-10_14:00:00
Reading /projects/mmc/SWIFTRegion/8Nov2013/WRF_144hrs/WRFdata/wrfout_d

In [10]:
xa.to_dataframe().head()

U10       V10          T2  \
Time                          height                                  
2013-11-06 12:00:00.003662109 0.0     0.40989 -3.397831  275.264038   
                              5.0     0.40989 -3.397831  275.264038   
                              10.0    0.40989 -3.397831  275.264038   
                              15.0    0.40989 -3.397831  275.264038   
                              20.0    0.40989 -3.397831  275.264038   

                                             TSK      UST          PSFC  \
Time                          height                                      
2013-11-06 12:00:00.003662109 0.0     274.728882  0.27731  90853.046875   
                              5.0     274.728882  0.27731  90853.046875   
                              10.0    274.728882  0.27731  90853.046875   
                              15.0    274.728882  0.27731  90853.046875   
                              20.0    274.728882  0.27731  90853.046875   

                                            HFX        LH           MUU  \
Time                          height                                      
2013-11-06 12:00:00.003662109 0.0    -14.032852  4.872955  80763.132812   
                              5.0    -14.032852  4.872955  80763.132812   
                              10.0   -14.032852  4.872955  80763.132812   
                              15.0   -14.032852  4.872955  80763.132812   
                              20.0   -14.032852  4.872955  80763.132812   

                                               MUV  ...  RU_TEND_ADV  \
Time                          height                ...                
2013-11-06 12:00:00.003662109 0.0     80755.640625  ...     0.006714   
                              5.0     80755.640625  ...     0.292037   
                              10.0    80755.640625  ...     0.842299   
                              15.0    80755.640625  ...     1.295481   
                              20.0    80755.640625  ...     1.964480   

                                      RU_TEND_PGF  RU_TEND_COR  RU_TEND_PHYS  \
Time                          height                                           
2013-11-06 12:00:00.003662109 0.0       66.278999   -14.339102    -38.899025   
                              5.0       69.043326   -19.429755    -31.578338   
                              10.0      59.872710   -24.687571    -23.913889   
                              15.0      57.942405   -26.890397    -24.519225   
                              20.0     104.336796   -29.521292    -21.352446   

                                        RV_TEND  RV_TEND_ADV  RV_TEND_PGF  \
Time                          height                                        
2013-11-06 12:00:00.003662109 0.0     45.427250    -3.068443    28.706203   
                              5.0     17.045369    -4.288580    -0.041901   
                              10.0    -8.752732    -6.283154   -25.770571   
                              15.0    31.359257    -7.153849    14.244196   
                              20.0    23.959743    -7.510135     7.143974   

                                      RV_TEND_COR  RV_TEND_PHYS  T_TEND_ADV  
Time                          height                                         
2013-11-06 12:00:00.003662109 0.0       -1.791386     21.580963   -2.690586  
                              5.0       -2.268481     23.644505   -3.768990  
                              10.0      -2.726643     26.027914   -4.718890  
                              15.0      -2.846576     27.115827   -5.162195  
                              20.0      -2.918640     27.244956   -5.762669  

[5 rows x 26 columns]

## Process data

In [11]:
# Round timestamp to 10min
xa['Time'] = xa['Time'].dt.round('10min')

Rescale tendencies with MU*

In [12]:
fieldnames_muu = ['RU_TEND','RU_TEND_ADV','RU_TEND_PGF','RU_TEND_COR','RU_TEND_PHYS']
fieldnames_muv = ['RV_TEND','RV_TEND_ADV','RV_TEND_PGF','RV_TEND_COR','RV_TEND_PHYS']
fieldnames_mut = ['T_TEND_ADV',]
for field in fieldnames_muu:
    xa[field].values = xa[field].values/xa['MUU'].values[:,np.newaxis]
for field in fieldnames_muv:
    xa[field].values = xa[field].values/xa['MUV'].values[:,np.newaxis]
for field in fieldnames_mut:
    xa[field].values = xa[field].values/xa['MUT'].values[:,np.newaxis]

Compute additional surface parameters

In [13]:
rho = xa['PSFC'] / (R_air*xa['T2'])

# Kinematic heat flux
xa['wt']  = xa['HFX'] / (Cp_air*rho)
xa['wt'].attrs['description'] = 'kinematic heat flux'
xa['wt'].attrs['units'] = 'K m s-1'

# Obukhov length
xa['L0']  = -xa['UST']**3 * xa['T2'] / (K * g * xa['wt'])
xa['L0'].attrs['description'] = 'Obukhov length'
xa['L0'].attrs['units'] = 'm'

Compute potential temperature from T2 and TSK

In [14]:
# Surface skin potential temperature
xa['thetaSK'] = xa['TSK'] * (P0/xa['PSFC'])**kappa
xa['thetaSK'].attrs['desccription'] = 'surface skin potential temperature'
xa['thetaSK'].attrs['units'] = 'K'

# Potential temperature at 2 m
xa['theta2'] = xa['T2'] * (P0/xa['PSFC'])**kappa
xa['theta2'].attrs['desccription'] = 'potential temperature at 2 m'
xa['theta2'].attrs['units'] = 'K'

Apply temporal averaging

In [15]:
Twindow = 3600. #Window size [s]

# Calculate number of samples in window
time_index = xa['Time'].values
tdelta = (time_index[1] - time_index[0]) / pd.Timedelta(1,unit='s')
windowsize = int(Twindow/tdelta) + 1
    
# Apply rolling average
xa_avg = xa.rolling(Time=windowsize,center=True,min_periods=1).mean()

In [16]:
# Save WRF column data
xa.to_netcdf('/projects/mmc/SWIFTRegion/8Nov2013/WRF_144hrs/SWIFT_L0w0.nc','w',format='NETCDF4')
xa_avg.to_netcdf('/projects/mmc/SWIFTRegion/8Nov2013/WRF_144hrs/SWIFT_L0w60.nc','w',format='NETCDF4')

## Write data as SOWFA input file

In [17]:
dateref = '2013-11-08 00:00:00'
datefrom = '2013-11-08 12:00:00'
dateto = '2013-11-09 12:00:00'

In [18]:
dirout = '/projects/mmc/SWIFTRegion/8Nov2013/WRF_144hrs/SWIFT_L0w0_v2'

In [19]:
# Generate dataframe with U, V, W, theta
df = xa.to_dataframe().loc[:,['U','V','W','theta']].reset_index()
df.rename(index=str, columns={"Time": "datetime"},inplace=True)
df.set_index('datetime',inplace=True)

# Write U, V, W, theta to fieldTable
timeheight_to_sowfa(dirout,'fieldTable',df,dateref,datefrom=datefrom,dateto=dateto,
                   xmom='U',ymom='V',zmom='W',temp='theta')

# Write initial profiles U, V, theta
ICs_to_sowfa(dirout,'initialValues',df,datefrom,xmom='U',ymom='V',temp='theta')

In [20]:
# Generate dataframe with U, V and theta tendencies
xa['FU'] = xa['RU_TEND_PGF'] + xa['RU_TEND_ADV']
xa['FV'] = xa['RV_TEND_PGF'] + xa['RV_TEND_ADV']

df = xa.to_dataframe().loc[:,['FU','FV','T_TEND_ADV']].reset_index()
df.rename(index=str, columns={"Time": "datetime"},inplace=True)
df.set_index('datetime',inplace=True)

# Write FU, FV and T_TEND_ADV to forcingTable
timeheight_to_sowfa(dirout,'forcingTable',df,dateref,datefrom=datefrom,dateto=dateto,
                   xmom='FU',ymom='FV',zmom='FW',temp='T_TEND_ADV') #FW does not exist so will be set to zero

In [22]:
# Generate dataframe with surface parameters
df = xa.to_dataframe().loc[:,['TSK','T2','thetaSK','theta2','wt']].reset_index()
df.rename(index=str, columns={"Time": "datetime"},inplace=True)
df.set_index('datetime',inplace=True)
df = df.loc[df.height==0].copy()

# Write surface parameters
BCs_to_sowfa(dirout,'surfaceSkinTemperatureTable',df,dateref,datefrom=datefrom,dateto=dateto,fieldname='TSK')
BCs_to_sowfa(dirout,'surface2mTemperatureTable',df,dateref,datefrom=datefrom,dateto=dateto,fieldname='T2')
BCs_to_sowfa(dirout,'surfaceSkinPotentialTemperatureTable',df,dateref,datefrom=datefrom,dateto=dateto,fieldname='thetaSK')
BCs_to_sowfa(dirout,'surface2mPotentialTemperatureTable',df,dateref,datefrom=datefrom,dateto=dateto,fieldname='theta2')
BCs_to_sowfa(dirout,'surfaceTemperatureFluxTable',df,dateref,datefrom=datefrom,dateto=dateto,fieldname='wt',fact=-1.0)